## Displaying Earth Engine products in web maps

In [1]:
import folium
from IPython.display import Image,display
import math

import ee
ee.Initialize()

%matplotlib inline

We are going to use the Landsat 8 sruface reflectance product as an example of creating web maps using Earth Engine. First, we have to grab the image collection

In [2]:
# getting the earth engine collection
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

Now, let's do some processing on the imagery to make a pretty map later on. This next block applies a cloud mask based on the internal Landsat 8 QA band.

In [3]:
# helper function to extract QA bits
def getQABits(image, start, end, newName):
    #Compute the bits we need to extract.
    pattern = 0;
    for i in range(start,end+1):
       pattern += math.pow(2, i);

    # Return a single band image of the extracted QA bits, giving the band
    # a new name.
    return image.select([0], [newName])\
                  .bitwiseAnd(int(pattern))\
                  .rightShift(start);

# function to mask clouds in LS SR products using internal QA flags
def maskClouds(image):
    clouds = getQABits(image.select('pixel_qa'),1,1,'qa')
    mask = clouds.neq(0)
    noClouds = image.updateMask(mask).set("system:time_start",image.get("system:time_start"))
    return noClouds

# apply cloud masking function
cloudFree = collection.map(maskClouds)

Next, we are going to calculate NDVI and then add it as a band to each image in the collection.

In [4]:
# function to add NDVI band to image collection
def calcNDVI(image):
    ndvi = image.normalizedDifference(['B5','B4'])
    return image.addBands(ndvi.rename(['ndvi']))

# apply NDVI calculation
ndviCollection = cloudFree.map(calcNDVI)

Now we are ready to create our composite image from a specific time period, in this case 2016-2017.

In [5]:
# create median composite image
composite = ndviCollection.filterDate('2016-01-01','2017-12-31').median()

### Creating the map tiles
#### RGB Composite Map
Here we get the web map tiles from Earth Engine by calling the "getMapId" method. Notice we provide the visualization parameters to the method. Here we define the min value, max value, bands to display, and gamma value.

Notice the Python syntax is a little different where the band values are not a list of strings but rather one string with the different arguments separated by a comma.

The "getMapId" method returns a dictionary with the mapid and a token values that we need to add to a URL string. The below example shows how this is done. The resulting URL string will be passed into the web mapping application for display.

In [6]:
# get map tile id and token with specific color palette
rgbmap = composite.getMapId({'min': 0, 'max': 3000, 'bands':'B4,B3,B2','gamma':1.3})
url = "https://earthengine.googleapis.com/map/"+rgbmap['mapid']+"/{z}/{x}/{y}?token="+rgbmap['token']
print(url)

https://earthengine.googleapis.com/map/0e3da1caf0337e965fc5554c1e037ef5/{z}/{x}/{y}?token=adf0943ab55d47cdda9edf426961721f


Now that we have our URL for the map tiles we can display the result Earth Engine data in a web map. Here we use Folium to display a Leaflet map and overlay the RGB composite Landsat image from Earth Engine.

In [7]:
map_ = folium.Map(location=[18.75, 99], zoom_start=6,tiles='cartodbpositron')

map_.add_tile_layer(tiles =url,name='True Color',attr='Google Earth Engine')
map_.add_child(folium.map.LayerControl())

map_

#### Single Band Map with Palette
Now that we successfully displayed a RGB composite image, we are now going to diaplay the composite NDVI image with a custom color palette. We use the same method as before, "getMapID", except here we select only the ndvi band and provide a palette key. Notice the value given the the palette key is a single string with the different arguments separated with a comma.

We are going to provide the mapid and token into the URL string again and display it on the web map application.


In [8]:
mapid = composite.select('ndvi').getMapId({'min': 0, 'max': 1,
              'palette' : '#dbdbdb,#ffffcc,#c2e699,#78c679,#31a354,#006837'})
ndviUrl = "https://earthengine.googleapis.com/map/"+mapid['mapid']+"/{z}/{x}/{y}?token="+mapid['token']
map__ = folium.Map(location=[18.75, 99], zoom_start=6,tiles='cartodbpositron')

map__.add_tile_layer(tiles =ndviUrl,name='NDVI',attr='Google Earth Engine')
map__.add_child(folium.map.LayerControl())
map__